# Zbx Clean Links

This notebook was created to perform semi-automatic cleaning of Zabbix database by interacting with files for mass data review and update.

### Environment setup

In [1]:
URL_ZABBIX = "http://zabbixsfi.anatel.gov.br/"
TOKEN_ZABBIX = "<PASTE SUPER ADMIN TOKEN HERE>"

HOST_DATA = ["hostids", "host", "name", "status"]
INTER_DATA = ["interfaceid","hostid","type","ip","dns","main"]
GROUP_DATA = ["groupid", "name"]

HOST_INTER_CSV = "zbxHostInterface.csv"
HOST_GROUP_CSV = "zbxHostGroup.csv"

CLEAN_DATA = "ZabbixClean.xlsx"
LINK_SHEET = "OutEnlace"
RFEYE_SHEET = "OutRFEye"
RMU_SHEET = "OutRMU"
OTHER_SHEET = "OutOutros"

### Includes

In [2]:
from pyzabbix import ZabbixAPI

import pandas as pd
import numpy as np

import json 

### Create session

In [3]:
zapi = ZabbixAPI(URL_ZABBIX)

zapi.session.verify = False # Disable SSL certificate verification

zapi.timeout = 5.1 #seconds

zapi.login(api_token=TOKEN_ZABBIX)

### Load host data

In [4]:
zbx_dict = zapi.host.get(output=HOST_DATA)

df_host = pd.DataFrame(zbx_dict)

### Load interface data

In [5]:
zbx_dict = zapi.hostinterface.get(output=INTER_DATA)

df_inter = pd.DataFrame(zbx_dict)

### Load template data

In [6]:
zbx_dict = zapi.template.get(output="extend")

df_template = pd.DataFrame(zbx_dict)

In [7]:
df_template

,proxy_hostid,host,status,lastaccess,ipmi_authtype,ipmi_privilege,ipmi_username,ipmi_password,maintenanceid,maintenance_status,...,tls_connect,tls_accept,tls_issuer,tls_subject,tls_psk_identity,tls_psk,proxy_address,auto_compress,custom_interfaces,uuid
0,0,Template OS Linux by Zabbix agent,3,0,-1,2,,,0,0,...,1,1,,,,,,1,0,f8f7908280354f2abeed07dc788c3747
1,0,Template App Zabbix Server,3,0,-1,2,,,0,0,...,1,1,,,,,,1,0,e2d2b4e4ac28483996cc11fe42823d57
2,0,Template App Zabbix Proxy,3,0,-1,2,,,0,0,...,1,1,,,,,,1,0,dd114bf0fb2f46bc84840f1bb24e2b23
3,0,Template Module Zabbix agent,3,0,-1,2,,,0,0,...,1,1,,,,,,1,0,8aa4557f6c3f4aadbc03447fca3af9f6
4,0,Template OS OpenBSD,3,0,-1,2,,,0,0,...,1,1,,,,,,1,0,760be6e1c2194a5bb7c0df47cc5f71ca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,0,Template Net MTR,3,0,-1,2,,,0,0,...,1,1,,,,,,1,0,7c6fab8b3b8048d685c1638702cc0ce4
131,0,ERMx FL SmartWeb,3,0,-1,2,,,0,0,...,1,1,,,,,,1,0,37390bd6e15c4251b229c913d5032b1f
132,0,Volt SmartWeb,3,0,-1,2,,,0,0,...,1,1,,,,,,1,0,51b6990a043447bdbcdbe1d275202b6d
133,0,CW RMU,3,0,-1,2,,,0,0,...,1,1,,,,,,1,0,dc41c7277a9f453bb2c088b45f7084fe


### Combine all data into a single table

Note that hosts with multiple interfaces will show up in multiple rows

In [8]:
df_full = pd.merge(df_host, df_inter, how='inner', left_on = 'hostid', right_on = 'hostid')

### Export host and interface data for external processing using CSV format

At this point one should export data from zabbix and manipulate to create a clean XLSX spreadsheet that shall be used for the new configuration

df_full.to_csv(HOST_INTER_CSV)

### Import clean data for links

In [8]:
df_link = pd.read_excel(CLEAN_DATA, sheet_name=LINK_SHEET)

### List all groups

In [9]:
zbx_dict = zapi.hostgroup.get(output=GROUP_DATA)

df_group = pd.DataFrame(zbx_dict)

### Create a dictionary with the groups to be processed

In [10]:
all_groups = {'GP Contrato':[],
              'GP Tecnologia de Acesso':[],
              'GP Situação do Enlace':[],
              'GP UF':[],}

for group in all_groups:
    np_array = df_link[group].unique()

    np_array = np_array[~pd.isna(np_array)] # remove NaN 

    all_groups[group] = np_array.tolist()

### Create groups in Zabbix if necessary and associate all data within the dataframe with clean data

In [11]:
for group_list in all_groups:
    
    zbx_group = {'groupid':[0]*all_groups[group_list].__len__(),
                 'name':[""]*all_groups[group_list].__len__()}
    i = 0
    
    for group in all_groups[group_list]:
        
        where_is_group = df_group[df_group.isin([group])].stack().index #get the index from where the group is in the df from zabbix
        
        if len(where_is_group) == 0: #check if the group don't exist in zabbix
            
            zbx_dict = zapi.hostgroup.create(name=group)

            zbx_group['groupid'][i] = zbx_dict['groupids']
            zbx_group['name'][i] = group
        else:
            index = where_is_group.to_numpy()[0][0]
            
            zbx_group['groupid'][i] = df_group['groupid'][index]
            zbx_group['name'][i] = df_group['name'][index]
            
        i = i + 1
        
    df_zbx_group = pd.DataFrame(zbx_group)
    
    df_link = pd.merge(df_link, df_zbx_group, how='left', left_on = group_list, right_on = 'name')
    
    df_link = df_link.rename(columns={"groupid": f"GID {group_list}", "name": f"zbx {group_list}"})

### Get hosts assigned to groups

In [12]:
def get_group_host(zapi, df_link):

    GROUP_DATA = ["groupid", "name"]
    
    df_output = pd.DataFrame(columns=['groupid', 'name', 'hostid'])

    for hostid in df_link["hostid"].to_list():
        zbx_dict = zapi.hostgroup.get(output=GROUP_DATA, hostids=hostid)
        
        df = pd.DataFrame(zbx_dict)
        df['hostid']=hostid
        
        df_output = pd.concat([df_output,df],ignore_index=True)
        
    return df_output

# df_host_group = get_group_host(zapi, df_link)
# df_host_group
# df_host_group.to_csv(HOST_GROUP_CSV)

### Create a list of dictionaries with groups to be removed

In [11]:
group_dic_list=[]

df = df_group.sort_values(by=["groupid"])
group_list = df['groupid'].to_list()[1:] # get group list except the first

for group in group_list:
    group_dic_list.append({'groupid':group})

### Create a list of dictionaries with host to be edited

In [12]:
host_dic_list=[]

host_list = df_link["hostid"].to_list()

for host in host_list:
    host_dic_list.append({'hostid':host})

### Select a reference default group

In [13]:
default_group = df['groupid'].to_list()[0]

group_List_dict = [{'groupid':default_group}]

### Add group 1 to all hosts to be edited

In [15]:
for host in host_list:
    zbx_dict = zapi.hostgroup.massadd(groups=default_group, hosts=[{'hostid':host}])

### Remove all groups from hosts, except default_group

In [16]:
for host in host_list:
    zbx_dict = zapi.hostgroup.massremove(groupids=group_list, hostids=host)

#### Add new groups from clean list

In [ ]:
def add_group(host):
    group_list = [{'groupid':host['GID GP Contrato']},
                  {'groupid':host['GID GP Tecnologia de Acesso']},
                  {'groupid':host['GID GP Situação do Enlace']},
                  {'groupid':host['GID GP UF']}]
    
    zbx_dict = zapi.hostgroup.massadd(groups=group_list, hosts=[{'hostid':host['hostid']}])
    
df_link.apply(add_group, axis=1)

### Remove default group

In [11]:
for host in host_list:
    zbx_dict = zapi.hostgroup.massremove(groupids=[default_group], hostids=[host])

### Change Host Visible Name 

In [18]:
def set_name(host):
    zbx_dict = zapi.host.update(hostid=host['hostid'], name=host['New Visible Name (name)'])
    
none = df_link.apply(set_name, axis=1)

### Remove all inventory data

In [35]:
CLEAN_INVENTORY = { 'alias' : '',
                    'asset_tag' : '',
                    'chassis' : '',
                    'contact' : '',
                    'contract_number' : '',
                    'date_hw_decomm' : '',
                    'date_hw_expiry' : '',
                    'date_hw_install' : '',
                    'date_hw_purchase' : '',
                    'deployment_status' : '',
                    'hardware' : '',
                    'hardware_full' : '',
                    'host_netmask' : '',
                    'host_networks' : '',
                    'host_router' : '',
                    'hw_arch' : '',
                    'installer_name' : '',
                    'location' : '',
                    'location_lat' : '',
                    'location_lon' : '',
                    'macaddress_a' : '',
                    'macaddress_b' : '',
                    'model' : '',
                    'name' : '',
                    'notes' : '',
                    'oob_ip' : '',
                    'oob_netmask' : '',
                    'oob_router' : '',
                    'os' : '',
                    'os_full' : '',
                    'os_short' : '',
                    'poc_1_cell' : '',
                    'poc_1_email' : '',
                    'poc_1_name' : '',
                    'poc_1_notes' : '',
                    'poc_1_phone_a' : '',
                    'poc_1_phone_b' : '',
                    'poc_1_screen' : '',
                    'poc_2_cell' : '',
                    'poc_2_email' : '',
                    'poc_2_name' : '',
                    'poc_2_notes' : '',
                    'poc_2_phone_a' : '',
                    'poc_2_phone_b' : '',
                    'poc_2_screen' : '',
                    'serialno_a' : '',
                    'serialno_b' : '',
                    'site_address_a' : '',
                    'site_address_b' : '',
                    'site_address_c' : '',
                    'site_city' : '',
                    'site_country' : '',
                    'site_notes' : '',
                    'site_rack' : '',
                    'site_state' : '',
                    'site_zip' : '',
                    'software' : '',
                    'software_app_a' : '',
                    'software_app_b' : '',
                    'software_app_c' : '',
                    'software_app_d' : '',
                    'software_app_e' : '',
                    'software_full' : '',
                    'tag' : '',
                    'type' : '',
                    'type_full' : '',
                    'url_a' : '',
                    'url_b' : '',
                    'url_c' : '',
                    'vendor' : ''}

def set_inventory(host):    
    zbx_dict = zapi.host.update(hostid=host['hostid'], inventory_mode='0', inventory=CLEAN_INVENTORY)
    
none = df_link.apply(set_inventory, axis=1)

### Add inventory data

In [36]:
def set_inventory(host):
    inventory = {'location_lat' : f"{host['Latitude']}",
                 'location_lon' : f"{host['Longitude']}",
                 'contract_number' : host['GP Contrato'],
                 'site_city' :       host['Tag Município'],
                 'site_state':       host['GP UF'],
                 'alias':            host['Tag Local']}
    
    zbx_dict = zapi.host.update(hostid=host['hostid'], inventory_mode='0', inventory=inventory)
    
none = df_link.apply(set_inventory, axis=1)

### Clean Tags

In [37]:
def set_tag(host):
    zbx_dict = zapi.host.update(hostid=host['hostid'], tags={})
    
none = df_link.apply(set_tag, axis=1)

### Add tag data

In [40]:
def set_tag(host):
    tag = [{'tag' : 'Local', 'value' : host['Tag Local']},
           {'tag' : 'Município', 'value' : host['Tag Município']}]
    
    zbx_dict = zapi.host.update(hostid=host['hostid'], tags=tag)
    
none = df_link.apply(set_tag, axis=1)